<a href="https://colab.research.google.com/github/vsky0/Gemini-API-General/blob/master/Gemini_Exercise_Apollo_11_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installation And Setup

Installing the required modules and importing.

In [1]:
!apt install tesseract-ocr libtesseract-dev
!pip install -q -U google-generativeai chromadb pytessaract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libarchive-dev libleptonica-dev tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  libarchive-dev libleptonica-dev libtesseract-dev tesseract-ocr tesseract-ocr-eng
  tesseract-ocr-osd
0 upgraded, 6 newly installed, 0 to remove and 45 not upgraded.
Need to get 8,560 kB of archives.
After this operation, 31.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 libarchive-dev amd64 3.6.0-1ubuntu1.1 [582 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libleptonica-dev amd64 1.82.0-3build1 [1,562 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 libtesseract-dev amd64 4.1.1-2.1build1 [1,600 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:5 http://arc

In [2]:
!apt install tesseract-ocr libtesseract-dev
!pip install -q -U google-generativeai chromadb pytesseract

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
libtesseract-dev is already the newest version (4.1.1-2.1build1).
tesseract-ocr is already the newest version (4.1.1-2.1build1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 64.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 273.8/273.8 kB 18.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.2/92.2 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━

Importing Modules

In [3]:
import time
from tqdm import tqdm

import pathlib

import google.generativeai as genai

import chromadb
from chromadb import Documents, EmbeddingFunction, Embeddings

import pandas as pd
from PIL import Image

import pytesseract

from IPython.display import Markdown

Load and Configure the API KEY

In [4]:
from google.colab import userdata
GOOGLE_API_KEY=userdata.get('G_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

Selecting the model, gemini-1.5-flash

In [5]:
model_name = 'gemini-1.5-flash'
model = genai.GenerativeModel(model_name)

# Gemini Final Excercise


## Data Preparation
Getting the resources from the link

In [6]:
!wget -O resources.zip "https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip"

--2024-07-27 11:53:05--  https://video.udacity-data.com/topher/2024/June/66744e79_resources/resources.zip
Resolving video.udacity-data.com (video.udacity-data.com)... 104.19.140.72, 104.19.138.72, 104.19.142.72, ...
Connecting to video.udacity-data.com (video.udacity-data.com)|104.19.140.72|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 286142532 (273M) [application/zip]
Saving to: ‘resources.zip’

resources.zip       100%[===================>] 272.89M   218MB/s    in 1.3s    

2024-07-27 11:53:06 (218 MB/s) - ‘resources.zip’ saved [286142532/286142532]



uzip the resources

In [7]:
!unzip resources.zip

Archive:  resources.zip
   creating: resources/
  inflating: __MACOSX/._resources    
   creating: resources/video/
  inflating: __MACOSX/resources/._video  
  inflating: resources/.DS_Store     
  inflating: __MACOSX/resources/._.DS_Store  
   creating: resources/audio/
  inflating: __MACOSX/resources/._audio  
   creating: resources/text/
  inflating: __MACOSX/resources/._text  
  inflating: resources/video/Apollo11PlaqueComparison.mov  
  inflating: __MACOSX/resources/video/._Apollo11PlaqueComparison.mov  
  inflating: resources/video/Apollo11Intro.mov  
  inflating: __MACOSX/resources/video/._Apollo11Intro.mov  
  inflating: resources/video/Apollo11MoonwalkMontage.mov  
  inflating: __MACOSX/resources/video/._Apollo11MoonwalkMontage.mov  
  inflating: resources/video/OneSmallStepCompilation.mov  
  inflating: __MACOSX/resources/video/._OneSmallStepCompilation.mov  
  inflating: resources/video/RaisingTheAmericanFlag.mov  
  inflating: __MACOSX/resources/video/._RaisingTheAmericanFl

In [8]:
data_dir = pathlib.Path("resources/")
all_file_names = [str(file) for file in data_dir.rglob("*") if file.is_file() and not file.name.startswith('.')]

In [9]:
for file_name in all_file_names:
    print(file_name)

print(len(all_file_names))

resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/text/images-023.jpg
resources/text/images-333.jpg
resources/text/images-020.jpg
resources/video/RaisingTheAmericanFlag.mov
resources/video/Apollo11MoonwalkMontage.mov
resources/video/Apollo11Intro.mov
resources/video/BuzzDescendsCompilation.mov
resources/video/OneSmallStepCompilation.mov
resources/video/Apollo11PlaqueComparison.mov
14


## Retrieval Augmented Generation - RAG

General Steps that involves a RAG are

1. Data Preparation
2. Datat Extraction and Summarization
3. Embedding Generation
4. Creating a Vector Database
5. Querying the RAG System

### Case: Text

Using the Tesseraccct OCR to extract text from images of the NASA report.

In [10]:
pytesseract.pytesseract.tesseract_cmd = (r'/usr/bin/tesseract')

Funcction extract the text from the images and summerazie them.

In [11]:
def create_text_summary():
  path = pathlib.Path("resources/text")

  text_summary_prompt = f"""You are an assistant tailored for summarizing text for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw text.
  Give a concise summary of the text that is well optimized for retrieval. Here is the text."""

  images = []
  text_summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue

    image = Image.open(f)
    response = model.generate_content(
                [text_summary_prompt, pytesseract.image_to_string(image)]
              )

    images.append(image)
    text_summaries.append(response.text)

  return images, text_summaries

altering the safety settings

In [12]:
safety_settings = [
    {
        "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
        "threshold": "BLOCK_NONE",
    },
]
model = genai.GenerativeModel('models/gemini-1.5-flash', safety_settings=safety_settings)


calling the function create_text_summary

In [13]:
image_files, text_summaries = create_text_summary()

checking the generated text summaries.

In [14]:
for text_summary in text_summaries:
  print(text_summary)


This document outlines the mission timeline for the Apollo 11 lunar landing, covering the launch, Earth orbit insertion, translunar coast, and lunar orbit insertion phases. It provides specific details on timing, events, and maneuvers, including launch window, TLI burn, CSM systems check, navigation sightings, and midcourse corrections. Table 1-1 provides additional data on burn details. 

This document outlines assumptions and ground rules for an EPS analysis. It covers battery usage during descent and ascent, S-band equipment uptime, rendezvous radar operation, navigation and guidance subsystem operation, and the state of forward window heaters. 

This document outlines the Apollo 11 mission plan, including operations and crew activities, for a July 16, 1969 launch.  The plan was prepared by the Flight Planning Branch with technical support from TRW Systems, and is under configuration control of the Crew Procedures Control Board (CPCB).  Changes to the plan must be submitted to the C

creating a chroma database using the generatd summaries.

In [15]:
class GeminiEmbeddingFunction(EmbeddingFunction):
  def __call__(self, input: Documents) -> Embeddings:
    model = 'models/text-embedding-004'
    title = 'Custome Query'
    return genai.embed_content(model = model,
                               content = input,
                               task_type = "retrieval_document",
                               title = title)["embedding"]


In [16]:
def create_chroma_db(documents, name):
    chroma_client = chromadb.Client()
    db = chroma_client.get_or_create_collection(name=name, embedding_function=GeminiEmbeddingFunction())

    for i, d in enumerate(documents):
      db.add(
          documents = d,
          ids = str(i)
      )
    return db

In [17]:
text_db = create_chroma_db(text_summaries, "text_nasa")

In [18]:
data = {
    'embeddings': text_db.peek()['embeddings'],
    'documents': text_db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.07339368760585785, -0.01851334050297737, 0....",This document outlines the mission timeline fo...
1,"[0.07816992700099945, 0.0014698952436447144, -...",This document outlines assumptions and ground ...
2,"[0.07204341888427734, 0.013148203492164612, 0....",This document outlines the Apollo 11 mission p...


function to fetch the relevant files.

In [19]:
def get_relevant_files(query, db):
  results = db.query(query_texts = [query], n_results = 3)
  return results["ids"][0]

In [20]:
files = get_relevant_files("Apollo 11 Fight Plan", text_db)
print(files)

['2', '0', '1']


### Case: Video and Audio

In [21]:
def create_summary(modality):
  path = data_dir / modality

  summary_prompt = f"""You are an assistant tailored for summarizing {modality} for retrieval.
  These summaries will be turned into vector embeddings and used to retrieve the raw {modality}.
  Give a concise summary of the {modality} that is well optimized for retrieval. Here is the {modality}."""

  files = []
  summaries = []

  for f in path.glob("*"):
    if f.is_dir() or f.name.startswith('.'):
      continue
    print(f)

    if modality == "text":
      file = Image.open(f)
      response = model.generate_content([summary_prompt, pytesseract.image_to_string(file)])

    else:
      file = genai.upload_file(f)

      while file.state.name == "PROCESSING":
        print("Waiting for video file upload...\n", end='')
        time.sleep(5)
        file = genai.get_file(file.name)

      response = model.generate_content([summary_prompt, file])

    files.append(file)
    summaries.append(response.text)

  return files, summaries

creating directory for all of the modalities

In [23]:
all_files = []
all_summaries = []

for modality_type in ["audio", "text", "video"]:
  files, summaries = create_summary(modality_type)
  all_files.extend(files)
  all_summaries.extend(summaries)

resources/audio/Apollo11OnboardAudioHighlightClip4.mp3
resources/audio/Apollo11OnboardAudioHighlightClip5.mp3
resources/audio/Apollo11OnboardAudioHighlightClip3.mp3
resources/audio/Apollo11OnboardAudioHighlightClip1.mp3
resources/audio/Apollo11OnboardAudioHighlightClip2.mp3
resources/text/images-023.jpg
resources/text/images-333.jpg
resources/text/images-020.jpg
resources/video/RaisingTheAmericanFlag.mov
Waiting for video file upload...
resources/video/Apollo11MoonwalkMontage.mov
Waiting for video file upload...
resources/video/Apollo11Intro.mov
Waiting for video file upload...
resources/video/BuzzDescendsCompilation.mov
Waiting for video file upload...
resources/video/OneSmallStepCompilation.mov
Waiting for video file upload...
resources/video/Apollo11PlaqueComparison.mov
Waiting for video file upload...


In [24]:
db = create_chroma_db(all_summaries, "nasa")

In [25]:
data = {
    'embeddings': db.peek()['embeddings'],
    'documents': db.peek()['documents']
}

df = pd.DataFrame.from_dict(data, orient='index').transpose()
df

,embeddings,documents
0,"[0.028120413422584534, 0.015564224682748318, 0...",Two people are talking about a flight plan. Th...
1,"[-0.02224438264966011, 0.006060068029910326, -...",Two people are discussing the readings on a ma...
2,"[0.0317809171974659, -0.018506387248635292, 0....","Footage of the Apollo 11 mission, including th..."
3,"[0.030713876709342003, -0.03630632534623146, -...",Buzz Aldrin steps off the Apollo 11 Lunar Modu...
4,"[-0.0008776529575698078, -0.031158626079559326...",This video shows Neil Armstrong descending the...
5,"[0.05018305033445358, -0.007730444427579641, -...",This video compares the original 1969 broadcas...
6,"[0.054390083998441696, -0.025909429416060448, ...",A group of astronauts are discussing the best ...
7,"[-0.007173345889896154, 0.01286245509982109, -...",Two people are discussing using a camera and m...
8,"[0.03270471468567848, -0.014274130575358868, -...",This audio is a conversation about a rocket la...
9,"[0.058798182755708694, 0.022343898192048073, 0...","Apollo 11 mission description: Launch, Earth o..."


In [26]:
files = get_relevant_files("Communication with Mission Control",db)
print(files)

['4', '7', '0']


function to query the RAG system

In [27]:
def query_rag(query, db):
  files = get_relevant_files(query, db)
  prompt = [all_files[int(f)] for f in files]
  prompt.append("Generate a response to the query using the provided files. Here's the query.")
  prompt.append(query)
  return model.generate_content(prompt).text, [all_file_names[int(f)] for f in files]

Querying the RAG system.

In [28]:
for response in query_rag("Explain what happened with Apollo 11 Mission.", db):
  print(response)

Apollo 11 was the spaceflight that landed the first humans on the moon. The mission launched on July 16, 1969 and landed on July 20, 1969.  Astronauts Neil Armstrong and Buzz Aldrin landed the lunar module Eagle on the surface of the moon.  They spent 21 hours, 36 minutes outside of the lunar module before returning to Earth.  The astronauts and lunar module successfully splashed down in the Pacific Ocean on July 24.  The mission marked a huge win for the United States in the space race against the Soviet Union. 

['resources/video/Apollo11Intro.mov', 'resources/text/images-023.jpg', 'resources/video/Apollo11MoonwalkMontage.mov']


In [29]:
for response in query_rag("What is the Translunar Coast. Explain in detail.", db):
  print(response)

The Translunar Coast is the period after the spacecraft is in a free lunar return trajectory after the TLI burn. This is the period in which the spacecraft is traveling from the Earth to the Moon.  The major events that occur during this phase are:
- Transposition, docking and LM ejection, including SIVB photography.
- Separation from SIVB and a CSM evasive maneuver.
- SIVB propulsive venting of propellants (slingshot).
- Two series of P23 cis-lunar navigation sightings, star/earth horizon, consisting of five sets at 06:00 GET and five sets at 24:30 GET.
- Four midcourse corrections which take place at TLI + 9, TLI + 24, LOI - 22 and LOI - 5 hours with ΔV nominally zero. 

['resources/text/images-023.jpg', 'resources/audio/Apollo11OnboardAudioHighlightClip3.mp3', 'resources/video/BuzzDescendsCompilation.mov']


In [30]:
import textwrap

In [31]:
# function to markdown the response of gemini
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [33]:
# ask more questions about Apollo 11
for response in query_rag("what programming language is used for appllo 11 mission, how much code is written.",db):
  print((response))

The information provided does not contain the programming languages and amount of code written for the Apollo 11 mission. 

['resources/text/images-020.jpg', 'resources/video/Apollo11Intro.mov', 'resources/text/images-023.jpg']
